In [ ]:
# Interactive Figure
# %matplotlib ipympl 
# Non interactive
%matplotlib inline 

In [ ]:
import time
import numpy as np
from matplotlib import pyplot as plt
from qualang_tools.units import unit
u = unit(coerce_to_integer=True)
from qm.qua import *
from qm import QuantumMachinesManager
from qm import SimulationConfig
from qualang_tools.results import fetching_tool
from qualang_tools.loops import from_array
import config_00 as config
import warnings
warnings.filterwarnings("ignore")
msg=np.load("msg_1007.npz")
msgI=msg["x"]
msgQ=msg["y"]

In [ ]:
def addjob(qmprog, qm):
    # Add a QUA program to the OPX queue, which compiles it and executes it
    job = qm.queue.add(qmprog)
    # Wait for job to be loaded
    while job.status=="loading":
        print("Job is loading...")
        time.sleep(0.1)
    # Wait until job is running
    time.sleep(0.1)
    while job.status=="pending":
        q = job.position_in_queue()
        if q>0:
            print("Position in queue",q,end='\r')
        time.sleep(0.1)
    job=job.wait_for_execution()
    print("\nJob is running")
    return job

In [ ]:
# Connect to the cluster (run only once)
import QM_cluster
qmm = QuantumMachinesManager(host=QM_cluster.QM_Router_IP, cluster_name=QM_cluster.cluster_name)

# Get running QM instance

In [ ]:
# Get the QM reference (rerun every time the config is changed)
qm_list =  qmm.list_open_qms()
qm = qmm.get_qm(qm_list[0])
print(f"Connected to {qm.id}")

# Send Pulses and Demodulate them

In [ ]:
n_points = 2048
n_pulses = n_points*config.readout_len//config.pulse_len
with program() as prog:
    i = declare(int)
    n = declare(int)
    m = declare(int)
    I = declare(fixed)  # QUA variable for the measured 'I' quadrature
    Q = declare(fixed)  # QUA variable for the measured 'Q' quadrature
    I_st = declare_stream()  # Stream for the 'I' quadrature
    Q_st = declare_stream()  # Stream for the 'Q' quadrature
    
    with for_(i, 0, i < n_pulses, i + 1):
        play("pulse", "qubit")
        frame_rotation(0.25,'qubit')

    with for_(n, 0, n < n_points, n + 1):
        # Demodulate the signals to get the 'I' & 'Q' quadratures)
        measure(
            "readout",
            "scope",
            dual_demod.full("cos", "sin", I),
            dual_demod.full("minus_sin", "cos", Q),
        )
        # Save the 'I' & 'Q' quadratures to their respective streams
        save(I, I_st)
        save(Q, Q_st)

    with stream_processing():
        # Cast the data into a 1D vector, and store the results on the OPX processor
        I_st.buffer(n_points).save("I")
        Q_st.buffer(n_points).save("Q")


In [ ]:
# Run the code and fetch results
job = addjob(prog, qm)
res = fetching_tool(job, ['I','Q'])
I, Q = res.fetch_all()
print('Job done')

In [ ]:
# Plot
fig,ax=plt.subplots()
ax.plot(I,Q,'.')
ax.set_aspect('equal')

# Practice

## Exercise 1: IQ rotation
- Modify the code to obtain a square
- Modify the code to obtain four dots at the four summits of a square
 
## Exercise 2: Secret message decoding
- Change the emission loop to:

        Im = declare(fixed,value=msgI)
        Qm = declare(fixed,value=msgQ)    
    
        with for_(i, 0, i < 4, i + 1):
            with for_(m, 0, m < len(msgI), m + 1):  
                play("pulse" * amp(Im[m],0.,Qm[m],0.),"qubit")

- Adjust the parameters to decode the message

## Exercise 3: Message encoding
- Encode the logo from Alice and Bob and send it in the same way as the previous message